In [34]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

from flask_cors import CORS

import pandas as pd

import os
import requests
import io

import datetime

In [35]:
url='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master/GUI_in.csv'
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')

df['Dprio']=df['Dprio'].astype('float')
df['Actual Bookings']=df['Actual Bookings'].astype('float')
df['Ramp-up frontier']=df['Ramp-up frontier'].astype('float')
df['Ideal curve (80% LF)']=df['Ideal curve (80% LF)'].astype('float')
df['Ideal curve (100% LF)']=df['Ideal curve (100% LF)'].astype('float')
df['Phase-down frontier']=df['Phase-down frontier'].astype('float')

dep_dates = sorted(df['DepDate'].unique())
dtime = sorted(df['dtime'].unique())

url='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master/GUI_in2.csv'
s=requests.get(url).content
df2=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')

df2['Dprio']=df2['Dprio'].astype('float')
df2['NumPax']=df2['NumPax'].astype('float')
df2['SpoilageRisk']=df2['SpoilageRisk'].astype('float')
df2['SpillageRisk']=df2['SpillageRisk'].astype('float')
df2['Intensity_full']=df2['Intensity_full'].astype('float')
df2['Intensity_downweighted']=df2['Intensity_downweighted'].astype('float')

df2=df2.sort_values(by=['DepDate','Dprio'])

df2['SpoilageRisk']=df2['SpoilageRisk'].apply(lambda x: float(str(str(x).split('.')[0])+'.'+str(str(x).split('.')[1])[0:3]))
df2['SpillageRisk']=df2['SpillageRisk'].apply(lambda x: float(str(str(x).split('.')[0])+'.'+str(str(x).split('.')[1])[0:3]))
df2['Intensity_full']=df2['Intensity_full'].apply(lambda x: float(str(str(x).split('.')[0])+'.'+str(str(x).split('.')[1])[0:3]))
df2['Intensity_downweighted']=df2['Intensity_downweighted'].apply(lambda x: float(str(str(x).split('.')[0])+'.'+str(str(x).split('.')[1])[0:3]))

df2['DepDate']=df2['DepDate'].apply(lambda x: pd.to_datetime(x))
df2['IssueDate']=df2['DepDate']-df2['Dprio'].apply(lambda x: datetime.timedelta(x-1))
df2=df2.loc[df2['IssueDate']==pd.to_datetime('2019-04-11'),:]
df2['DepDate']=df2['DepDate'].astype('str')
df2['DepDate']=df2[[x for x in df2.columns if x!='IssueDate']]

df2=df2[['DepDate','dtime','SpoilageRisk','Intensity_downweighted','Intensity_full','SpillageRisk']]

In [36]:
url='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master/GUI_in3.csv'
s=requests.get(url).content
df3=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')

In [ ]:
df3['Dprio']=df3['Dprio'].astype('float')
df3['ActualRevenue_full']=df3['ActualRevenue_full'].astype('float')
df3['ActualRevenue_downweighted']=df3['ActualRevenue_downweighted'].astype('float')
df3['TotalRevenue_full']=df3['TotalRevenue_full'].astype('float')
df3['TotalRevenue_downweighted']=df3['TotalRevenue_downweighted'].astype('float')

# Import external stylesheet

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Layout

In [ ]:
line_columns_Pax = ['Actual Bookings', 'Ramp-up frontier', 'Ideal curve (80% LF)', 'Ideal curve (100% LF)', 'Phase-down frontier']
linecolor_dict_Pax = dict(zip(line_columns_Pax,['black', 'red', 'blue', 'green', '#E0115F']))
linedash_dict_Pax = dict(zip(line_columns_Pax,['solid', 'dot', 'solid', 'solid', 'dot']))
name_dict_Pax = dict(zip(line_columns_Pax,['Actual Bookings', 'Ramp-up frontier', 'Ideal curve (80% LF)', 'Ideal curve (100% LF)', 'Phase-down frontier']))

In [ ]:
line_columns_Revenue = ['ActualRevenue_full','TotalRevenue_full','TotalRevenue_downweighted']
linecolor_dict_Revenue = dict(zip(line_columns_Revenue,[ 'black','green', 'blue']))
linedash_dict_Revenue = dict(zip(line_columns_Revenue,['solid', 'solid', 'solid']))
name_dict_Revenue = dict(zip(line_columns_Revenue,['ActualRevenue_full','TotalRevenue_full','TotalRevenue_downweighted']))

In [ ]:
rename_dict=dict(zip(df2.columns,
['Departure Date', 'Departure Time', 'Spoilage Risk', 'Intensity (80% LF)', 'Intensity (100% LF)', 'Spillage Risk']))

In [ ]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    
    # MAIN HEADER
    html.Div([html.H1(children='Booking Curve Steering System',
    style={
            'textAlign': 'center',
            'color': 'black',
            'margin-bottom': '2em',
            'padding' : '50px' ,
            'backgroundColor' : '#66b3cc',
            'width': '90.23%',
            'display': 'inline-block'
        }),html.Img(src=app.get_asset_url('LuxairGroup_logo.jpg'), 
              style = {'backgroundColor' : '#66b3cc', 
                       'width': '5%', 
                       'height': '6em', 
                       'display': 'inline-block',
                       'margin-left': '0em'
                      # 'margin-right': '1em'
                      })
                       
#               html.Div(style={'backgroundColor' : '#66b3cc',
#             'width': '50%',
#             'display': 'inline-block'})
        ]),
    
    # DROPDOWN MENUES
    
        # DROPDOWN MENU FOR DEPATURE DATE
    html.Div([html.Div([html.Label('Departure Date', style={'textAlign': 'center'}), 
    dcc.Dropdown(id = 'dropdown_depdates',
    options=[{
    'label': i,
    'value': i
    } for i in dep_dates],
    placeholder="Select a departure date",                                 
    value='All Departure Dates')],
    style={'width': '10%', 'display': 'inline-block', 'margin-bottom': '1.25em', 'margin-left': '50em'}),
        
        # DROPDOWN MENU FOR DEPARTURE TIME
    html.Div([html.Label('Departure Time', style={'textAlign': 'center'}),
    dcc.Dropdown(id = 'dropdown_dtime',
    options=[{
    'label': i,
    'value': i
    } for i in dtime],
    placeholder="Select a departure time",                       
    value='All Departure Times')],
    style={'width': '10%', 'display': 'inline-block', 'margin-bottom': '1.25em', 'margin-left': '2.5em'})        
             ]),
    
    # CONTAINERS
    
        # TABLE CONTAINER
    html.Div([html.Div([html.H3(children='Risk Table',style={
             'textAlign': 'center',
             'color': 'black',
             'margin-bottom': '0.75em',
         }),
        dash_table.DataTable(
        id = 'booking_table',
        data=df2.to_dict('rows'),
        columns=[{'id': c, 'name': rename_dict[c]} for c in df2.columns],
        style_header={'backgroundColor': 'rgb(30, 30, 30)','color': 'white', 'textAlign': 'center'},

        style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(40, 40, 40)',
            'color': 'white',
            'textAlign': 'center'
        },
        {
            'if': {'row_index': 'odd', 'column_id': 'Dprio'},
            'backgroundColor': '#568dba',
            'color': 'black',
            'textAlign': 'center'
        },    
        {
            'if': {'row_index': 'even'},
            'backgroundColor': 'rgb(50, 50, 50)',
            'color': 'white',
            'textAlign': 'center'
        },
        {
            'if': {'row_index': 'even', 'column_id': 'Dprio'},
            'backgroundColor': '#6798c1',
            'color': 'black',
            'textAlign': 'center'
        },     
        {
             'if': {
                 'column_id': 'SpoilageRisk',
                 'filter': 'SpoilageRisk >= num(0.95)',
             },
            'color': 'red',
        },
        {
            'if': {
                 'column_id': 'SpillageRisk',
                 'filter': 'SpillageRisk >= num(0.95)',
             },
            'color': '#E0115F'
        }
        ],
        style_cell={
        # all three widths are needed
        'minWidth': '50px', 'width': '50px', 'maxWidth': '50px',
        'whiteSpace': 'normal'
    },
        
    sorting = True,
    sorting_type = "multi",
    filtering = True
          
        )], style={'width': '40%', 'display': 'inline-block'}),
    
        # GRAPH CONTAINER                   
    html.Div([html.H3(children='Booking Curves',style={
             'textAlign': 'center',
             'color': 'black',
             'margin-bottom': '0.75em',
         }),
              dcc.Graph(id='graph-container_1'),
              dcc.Graph(id='graph-container_2')],
             style={'width': '60%', 'float': 'right', 'display': 'inline-block'})]),
])

In [ ]:
@app.callback(
    # Output('table-container', 'children'),
    Output('graph-container_1', 'figure'),
    [Input('dropdown_depdates', 'value'),
    Input('dropdown_dtime', 'value')])
def update_graph(value_depdates, value_dtime):

    dff = df.loc[(df['DepDate'] == value_depdates)&(df['dtime'] == value_dtime),:] # update with your own logic    
    
    traces = []
    for i in line_columns_Pax:
        traces.append(go.Scatter(
            x=dff['Dprio'].tolist(),
            y=dff[i].tolist(),
            mode = 'lines',
            name = name_dict_Pax[i],
            line = dict(
              dash = linedash_dict_Pax[i],
              color = linecolor_dict_Pax[i],
              width = 2
           )
    ))
   
    return {
    'data': traces,
    'layout': go.Layout(
        xaxis={'title': 'Days prior to departure', 'range': [dff['Dprio'].min(), 365]},
        yaxis={'title': 'Number of bookings', 'range': [0, max([dff['Phase-down frontier'].max(),dff['Actual Bookings'].max()])]},
        height=650)
    }

In [ ]:
@app.callback(
    # Output('table-container', 'children'),
    Output('graph-container_2', 'figure'),
    [Input('dropdown_depdates', 'value'),
    Input('dropdown_dtime', 'value')])
def update_graph(value_depdates, value_dtime):

    dff = df3.loc[(df3['DepDate'] == value_depdates)&(df3['dtime'] == value_dtime),:] # update with your own logic    
    
    traces = []
    for i in line_columns_Revenue:
        traces.append(go.Scatter(
            x=dff['Dprio'].tolist(),
            y=dff[i].tolist(),
            mode = 'lines',
            name = name_dict_Revenue[i],
            line = dict(
              dash = linedash_dict_Revenue[i],
              color = linecolor_dict_Revenue[i],
              width = 2
           )
    ))
   
    return {
    'data': traces,
    'layout': go.Layout(
        xaxis={'title': 'Days prior to departure', 'range': [dff['Dprio'].min(), 365]},
        yaxis={'title': 'Number of bookings', 'range': [min([dff['ActualRevenue_downweighted'].min(),dff['TotalRevenue_downweighted'].min()]), 
                                                        max([dff['ActualRevenue_full'].max(),dff['TotalRevenue_full'].max()])]},
        height=650)
    }

In [ ]:
if __name__ == '__main__':
    app.run_server(port=1000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:1000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Apr/2019 21:51:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 21:51:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 21:51:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 21:51:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 21:51:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 21:51:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 21:51:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 21:51:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 21:52:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 21:52:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 21:53:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 